# Long-term Memory
This notebook demonstrates how to take advantage of RAG-like applications to build a memory that is persisted across sessions.

## What we'll learn:
- Basic ideas of registering and searching memory
- Create abstraction to handle long-term memory
- Bind long-term memory to your agents

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
from typing import List, Dict, Optional
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from dotenv import load_dotenv

from lib.vector_db import VectorStoreManager
from lib.documents import Document, Corpus
from lib.agents import Agent
from lib.tooling import Tool

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Playing with Vector DB

In [5]:
db = VectorStoreManager(OPENAI_API_KEY)
vector_store = db.get_or_create_store("test")

In [6]:
vector_store.add(
    Document(
        content= "I prefer Nintendo games", 
        metadata = {
            "user_id": "1", 
            "session_id": "games",
            "timestamp": datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
        },
    )
)

In [7]:
vector_store.add(
    Corpus([
        Document(
            content= "I prefer Sony games", 
            metadata = {
                "user_id": "2", 
                "session_id": "games",
                "timestamp": datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
            },
        ),
        Document(
            content= "I have an Electric Car", 
            metadata = {
                "user_id": "2", 
                "session_id": "vehicles",
                "timestamp": datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
            },
        )
    ])
)

In [8]:
vector_store.query(
    query_texts=["game preference"],
    n_results=1,
    where={"user_id": "2"},
)

{'ids': [['04cee36b-093a-4bab-923b-15c4e30de250']],
 'embeddings': None,
 'documents': [['I prefer Sony games']],
 'uris': None,
 'included': ['documents', 'distances', 'metadatas'],
 'data': None,
 'metadatas': [[{'session_id': 'games',
    'timestamp': '06-22-2025 23:22:51',
    'user_id': '2'}]],
 'distances': [[0.3349607586860657]]}

## Useful Abstractions

- MemoryFragment
- MemorySearchResult
- TimestampFilter
- LongTermMemory

In [9]:
@dataclass
class MemoryFragment:
    """
    Represents a single piece of memory information stored in the long-term memory system.
    
    This class encapsulates user preferences, facts, or contextual information that can be
    retrieved later to provide personalized responses in conversational AI applications.
    
    Attributes:
        content (str): The actual memory content or information to be stored
        owner (str): Identifier for the user who owns this memory fragment
        namespace (str): Logical grouping for organizing related memories (default: "default")
        timestamp (int): Unix timestamp when the memory was created (auto-generated)
    """
    content: str
    owner: str 
    namespace: str = "default"
    timestamp: int = field(default_factory=lambda: int(datetime.now().timestamp()))

In [10]:
@dataclass
class MemorySearchResult:
    """
    Container for the results of a memory search operation.
    
    Encapsulates both the retrieved memory fragments and associated metadata
    such as distance scores from the vector search.
    
    Attributes:
        fragments (List[MemoryFragment]): List of memory fragments matching the search query
        metadata (Dict): Additional information about the search results (e.g., distances, scores)
    """
    fragments: List[MemoryFragment]
    metadata: Dict

In [11]:
@dataclass
class TimestampFilter:
    """
    Filter criteria for time-based memory searches.
    
    Allows filtering memory fragments based on when they were created,
    enabling retrieval of recent memories or memories from specific time periods.
    
    Attributes:
        greater_than_value (int, optional): Unix timestamp - only return memories created after this time
        lower_than_value (int, optional): Unix timestamp - only return memories created before this time
    """
    greater_than_value: int = None
    lower_than_value: int = None

In [12]:
class LongTermMemory:
    """
    Manages persistent memory storage and retrieval using vector embeddings.
    
    This class provides a high-level interface for storing and searching user memories,
    preferences, and contextual information across conversation sessions. It uses
    vector similarity search to find relevant memories based on semantic meaning.
    
    The memory system supports:
    - Multi-user memory isolation
    - Namespace-based organization
    - Time-based filtering
    - Semantic similarity search
    """
    def __init__(self, db:VectorStoreManager):
        self.vector_store = db.create_store("long_term_memory", force=True)

    def get_namespaces(self) -> List[str]:
        """
        Retrieve all unique namespaces currently stored in memory.
        
        Useful for understanding how memories are organized and for
        administrative purposes.
        
        Returns:
            List[str]: List of unique namespace identifiers
        """
        results = self.vector_store.get()
        namespaces = [r["metadatas"][0]["namespace"] for r in results]
        return namespaces

    def register(self, memory_fragment:MemoryFragment, metadata:Optional[Dict[str, str]]=None):
        """
        Store a new memory fragment in the long-term memory system.
        
        The memory is converted to a vector embedding and stored with associated
        metadata for later retrieval. Additional metadata can be provided to
        enhance searchability.
        
        Args:
            memory_fragment (MemoryFragment): The memory content to store
            metadata (Optional[Dict[str, str]]): Additional metadata to associate with the memory
        """
        complete_metadata = {
            "owner": memory_fragment.owner,
            "namespace": memory_fragment.namespace,
            "timestamp": memory_fragment.timestamp,
        }
        if metadata:
            complete_metadata.update(metadata)

        self.vector_store.add(
            Document(
                content=memory_fragment.content,
                metadata=complete_metadata,
            )
        )

    def search(self, query_text:str, owner:str, limit:int=3,
               timestamp_filter:Optional[TimestampFilter]=None, 
               namespace:Optional[str]="default") -> MemorySearchResult:
        """
        Search for relevant memories using semantic similarity.
        
        Performs a vector similarity search to find memories that are semantically
        related to the query text. Results are filtered by owner, namespace, and
        optionally by timestamp range.
        
        Args:
            query_text (str): The search query to find similar memories
            owner (str): User identifier to filter memories by ownership
            limit (int): Maximum number of results to return (default: 3)
            timestamp_filter (Optional[TimestampFilter]): Time-based filtering criteria
            namespace (Optional[str]): Namespace to search within (default: "default")
            
        Returns:
            MemorySearchResult: Container with matching memory fragments and metadata
        """

        where = {
            "$and": [
                {
                    "namespace": {
                        "$eq": namespace
                    }
                },
                {
                    "owner": {
                        "$eq": owner
                    }
                },
            ]
        }

        if timestamp_filter:
            if timestamp_filter.greater_than_value:
                where["$and"].append({
                    "timestamp": {
                        "$gt": timestamp_filter.greater_than_value,
                    }
                })
            if timestamp_filter.lower_than_value:
                where["$and"].append({
                    "timestamp": {
                        "$lt": timestamp_filter.lower_than_value,
                    }
                })

        result = self.vector_store.query(
            query_texts=[query_text],
            n_results=limit,
            where=where
        )

        fragments = []
        documents = result.get("documents", [[]])[0]
        metadatas = result.get("metadatas", [[]])[0]

        for content, meta in zip(documents, metadatas):
            owner = meta.get("owner")
            namespace = meta.get("namespace", "default")
            timestamp = meta.get("timestamp")

            fragment = MemoryFragment(
                content=content,
                owner=owner,
                namespace=namespace,
                timestamp=timestamp
            )

            fragments.append(fragment)
        
        result_metadata = {
            "distances": result.get("distances", [[]])[0]
        }

        return MemorySearchResult(
            fragments=fragments,
            metadata=result_metadata
        )


In [13]:
ltm = LongTermMemory(db)

In [14]:
now = datetime.now()
past_7d = (now - timedelta(days=7)).timestamp()
past_10d = (now - timedelta(days=10)).timestamp()
past_14d = (now - timedelta(days=14)).timestamp()

In [15]:
memories = [
    MemoryFragment(
        content="I prefer dark mode", 
        timestamp=past_7d, 
        owner="Henrique"
    ),
    MemoryFragment(
        content="I have a Nintendo Switch", 
        timestamp=past_10d, 
        owner="Henrique"
    ),
    MemoryFragment(
        content="I drove an electric car yesterday", 
        timestamp=past_14d, 
        owner="Henrique"
    ),
]

In [16]:
for m in memories:
    ltm.register(m)

In [17]:
ltm.search(
    query_text="What are my ligthing preferences?",
    owner="Henrique",
    limit=1,
)

MemorySearchResult(fragments=[MemoryFragment(content='I prefer dark mode', owner='Henrique', namespace='default', timestamp=1750040572.504482)], metadata={'distances': [0.37668654322624207]})

In [18]:
result = ltm.search(
    query_text=" ",
    owner="Henrique",
    timestamp_filter=TimestampFilter(
        greater_than_value=past_14d,
        lower_than_value=past_7d
    ),
    limit=5,
)
print(result.fragments)
print(result.metadata)

[MemoryFragment(content='I have a Nintendo Switch', owner='Henrique', namespace='default', timestamp=1749781372.504482)]
{'distances': [0.5204347372055054]}


## Agent

In [19]:
def build_memory_registration_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to register new memories.
    
    This factory function creates a tool that allows AI agents to store new
    information about users in the long-term memory system. The tool is
    pre-configured with specific owner and namespace parameters.
    
    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace for organizing memories
        
    Returns:
        Tool: A configured tool for memory registration
    """
    def _register(content:str):
        ltm.register(
            MemoryFragment(
                content=content, 
                owner=owner,
                namespace=namespace
            )
        )
        return "Saved new memory"

    return Tool(
        func=_register, 
        name="register_memory", 
        description=(
            "Register a new memory or preference about the user, " 
            "so it can be useful later as context.\n"
            "Args:\n"
            "    content: The information to save"
        )
    )

In [20]:
def build_memory_search_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to search existing memories.
    
    This factory function creates a tool that allows AI agents to retrieve
    relevant memories from the long-term memory system based on semantic
    similarity to a search query.
    
    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace to search within
        
    Returns:
        Tool: A configured tool for memory search
    """
    def _search(content:str):
        result = ltm.search(
            query_text=content,
            owner=owner,
            namespace=namespace,
            limit=3,
        )
        return str(tuple(zip(result.fragments, result.metadata['distances'])))

    return Tool(
        func=_search, 
        name="search_memory", 
        description=(
            "Search for a stored memory or preference about the user, " 
            "so it's useful as a context.\n"
            "Args:\n"
            "    content: The information to look for"
        )
    )

In [21]:
ltm = LongTermMemory(db)

In [22]:
agent = Agent(
    model_name="gpt-4o-mini",
    tools=[
        build_memory_registration_tool(ltm, "Henrique", "conversation"),
        build_memory_search_tool(ltm, "Henrique", "conversation")
    ],
    instructions=(
        "You are a helpful assistant. Try to use memory if needed. " 
        "And if the user shares a preference, use your tools to register memories."
    )
)

In [23]:
result = agent.invoke(
    query="I prefer dark mode",
    session_id="session_1"
)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [24]:
result.get_final_state()

{'user_query': 'I prefer dark mode',
 'instructions': 'You are a helpful assistant. Try to use memory if needed. And if the user shares a preference, use your tools to register memories.',
 'messages': [SystemMessage(role='system', content='You are a helpful assistant. Try to use memory if needed. And if the user shares a preference, use your tools to register memories.'),
  UserMessage(role='user', content='I prefer dark mode'),
  AIMessage(role='assistant', content=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vkL7gPEh6a8509kcX0rloOgG', function=Function(arguments='{"content":"User prefers dark mode."}', name='register_memory'), type='function')]),
  ToolMessage(role='tool', content='"Saved new memory"', tool_call_id='call_vkL7gPEh6a8509kcX0rloOgG', name='register_memory'),
  AIMessage(role='assistant', content="I've noted that you prefer dark mode! If there's anything else you'd like to share or if you have more preferences, feel free to let me know.", tool_calls=None)],


In [25]:
result = agent.invoke(
    query="What are my lighting preferences?",
    session_id="session_2"
)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [26]:
result.get_final_state()

{'user_query': 'What are my lighting preferences?',
 'instructions': 'You are a helpful assistant. Try to use memory if needed. And if the user shares a preference, use your tools to register memories.',
 'messages': [SystemMessage(role='system', content='You are a helpful assistant. Try to use memory if needed. And if the user shares a preference, use your tools to register memories.'),
  UserMessage(role='user', content='What are my lighting preferences?'),
  AIMessage(role='assistant', content=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Bd4Hd3bJhI8zHkQ1x7PlOAAf', function=Function(arguments='{"content":"lighting preferences"}', name='search_memory'), type='function')]),
  ToolMessage(role='tool', content='"((MemoryFragment(content=\'User prefers dark mode.\', owner=\'Henrique\', namespace=\'conversation\', timestamp=1750645374), 0.32105663418769836),)"', tool_call_id='call_Bd4Hd3bJhI8zHkQ1x7PlOAAf', name='search_memory'),
  AIMessage(role='assistant', content="Your ligh